# Project Part 2 Solutions

## (a)

In [1]:
import numpy as np

file=np.load("part2.npz")
beta_old=file['beta_old']
N=file['N']
Svc_0_PMF=file['Svc_0_PMF']
Lc=file['Lc']
Ic_0=file['Ic_0']
gamma=file['gamma']
L_observed=file['L_observed']
nb_nodes = Svc_0_PMF.shape[0]

In [2]:
def SIR(t, Z, N, betas, gamma):
    (S_vec, I_vec, R) = (Z[:16].reshape(4, 4), Z[16:20], Z[-1])
    I = I_vec.sum()
    dSdt = -betas*S_vec*I/N 
    dIdt = -dSdt.sum(axis=0)-gamma*I_vec
    dRdt = np.array([gamma*I])
    return np.concatenate((dSdt.flatten(), dIdt, dRdt))

In [57]:
from scipy.integrate import solve_ivp
from scipy.optimize import fmin
from sklearn.metrics import mean_squared_error

Svc_0_PMF /= Svc_0_PMF.sum(axis=(1, 2), keepdims=True)
Svc_0 = (N-Ic_0.sum(axis=1))[:, np.newaxis, np.newaxis]*Svc_0_PMF
Svc_0 = Svc_0.round().astype(int)
R0 = np.zeros((nb_nodes)).astype(int)

t = np.linspace(0, 199, 200)

S = np.zeros((nb_nodes, 4, 4, len(t)))
I = np.zeros((nb_nodes, 4, len(t)))
R = np.zeros((nb_nodes, len(t)))
L = np.zeros((nb_nodes, len(t)))

for N_node, node_index in zip(N, range(nb_nodes)):

    initial_conditions = np.concatenate(
        (Svc_0[node_index].flatten(), Ic_0[node_index], [R0[node_index]]))

    soln = solve_ivp(
        SIR, (t[0], t[9]), initial_conditions, args=(N_node, beta_old, gamma), t_eval=t[:10])

    S[node_index, :, :, :10] = soln.y[:16].reshape(4, 4, -1)
    I[node_index, :, :10] = soln.y[16:20]
    R[node_index, :10] = soln.y[20]
    L[node_index, :10] = Lc@soln.y[16:20]


alpha_predicted = np.zeros((nb_nodes))


def loss_func(alpha):
    initial_conditions = np.concatenate(
        (S[node_index, :, :, 9].flatten(), I[node_index, :, 9], [R[node_index, 9]]))
    
    soln = solve_ivp(
        SIR, (t[10], t[29]), initial_conditions, args=(N_node, beta_old*alpha, gamma), t_eval=t[10:30])
    
    S[node_index, :, :, 10:30] = soln.y[:16].reshape(4, 4, -1)
    I[node_index, :, 10:30] = soln.y[16:20]
    R[node_index, 10:30] = soln.y[20]
    L[node_index, 10:30] = Lc@soln.y[16:20]
    
    mse = mean_squared_error(L_observed[node_index, 10:30], L[node_index, 10:30])
    

    return mse


for N_node, node_index in zip(N, range(nb_nodes)):
    alpha_predicted[node_index] = fmin(loss_func, 0.5, disp=False)


In [59]:
alpha_predicted

array([0.22669983, 0.39484253, 0.22185059, 0.73707123, 0.70143738,
       0.20029602, 0.35754395, 0.35007172, 0.3601532 , 0.16713867,
       0.27568665, 0.32045898, 0.31580811, 0.20931396, 0.12352142,
       0.22246094, 0.18051758, 0.56386719, 0.59924927, 0.14249268,
       0.26811523, 0.27178955, 0.49281006, 0.26390381, 0.33642883,
       0.15171814, 0.3757782 , 0.21018066, 0.44144897, 0.68422394,
       0.29797363, 0.3010498 , 0.39297028, 0.16387329, 0.61907349,
       0.66160202, 0.3225708 , 0.15137939, 0.28189697, 0.90992126,
       0.41170807, 0.43126221, 0.43154907, 0.17045288, 0.24889221,
       0.41756592, 0.71300049, 0.330896  , 0.23173828, 0.20965576,
       0.13822021, 0.20452881, 0.33418884, 0.26639404, 0.14807129,
       0.66747131, 0.17644043, 0.53582764, 0.27561264, 0.56835938,
       0.36387939, 0.27543945, 0.34067383, 0.42473755, 0.24645996,
       0.52313232, 0.20018311, 0.38779907, 0.46865234, 0.1552803 ,
       0.14443359, 0.33525391, 0.2932373 , 0.44838867, 0.41690

## (b)

In [ ]:
def g(money):
    alpha = 1/np.log2(2*10**(-3)*money+2)
    return alpha

def g_inv(alpha):
    money = (2**(1/alpha)-2)/(2*10**(-3))
    return money
    import matplotlib.pyplot as plt 

def scatter_plot(alpha_afterNPI):
    plt.figure()
    colors=(beta_old[np.newaxis,:,:]*Svc_0_PMF).mean(axis=(1,2))
    plt.scatter(alpha_predicted, alpha_afterNPI, s=N/500, c=colors, cmap='jet', alpha=0.5)
    plt.xlabel(r"$\alpha_{own}$")
    plt.ylabel(r"$\alpha_{after\ NPI}$")

def total_recovered_difference(alpha_afterNPI):
    
    ###### WRITE YOUR FUNCTION HERE
    
    return difference

### Policy #1

i.

ii.

### Policy #2

i. 

ii.

### Policy #3

i.

ii.

### Policy #4

i.

ii.

## (c)

### Policy 1

### Policy 2

### Policy 3

### Policy 4